In [1]:
from python_storybook import core

In [2]:
render = core.Story(
    story_name="Example",
    story="Hi"
)

In [133]:
def get_docs(func):
    import inspect
    return inspect.getdoc(func)

def get_type_hints_inspect(func):
    import inspect
    
    signature = inspect.signature(func)
    type_hints = {name: param.annotation for name, param in signature.parameters.items()}
    type_hints["return"] = signature.return_annotation
    return type_hints

def get_type_hints_typing(func):
    from typing import get_type_hints
    type_hints = get_type_hints(func)
    return type_hints

def get_type_hints(func):
    type_hints = get_type_hints_typing(func)
    type_hints = {key: value.__name__ for key, value in type_hints.items()}
    return type_hints

In [137]:
get_type_hints(render)


{'arg1': int, 'arg2': int, 'return': int}

In [138]:
int, int.__name__

(int, 'int')

In [207]:
from pydantic import BaseModel
from typing import List, Any, Optional, Callable, Dict, Union
import os

class Story(BaseModel):
    name: str
    render: Callable[[Any], str]
    meta: Optional[Any]
    parent: str

class StoryMeta(BaseModel):
    title:str
    name:str
    render_name:str
    docs: str
    type_hints:Dict[str, str]

class StoryManager():
    def __init__(self, title: str):
        self.title = title
        self.meta: Any = ""
        self.stories: List[Story] = []
        # self.path: str = os.path.abspath(__file__)
        Head.register(story_manager=self)

    def create_story(self, story_name: str, render: Callable[[Any], str])->None:
        story = Story(
            name=story_name,
            meta=self.meta,
            render=render,
            parent=self.title
        )
        self.stories.append(
            story
        )

    def get_story(self, story_name:str)->Story:
        for story in self.stories:
            if story_name==story.name:
                return story
        raise f"There is no story named {story_name}"

    def get_story_names(self)->List[str]:
        return [story.name for story in self.stories]


class Head():
    _all_managers:List[StoryManager] = []

    @staticmethod
    def register(story_manager: StoryManager)->None:
        if story_manager.title in Head.get_titles():
            raise "Title can't be duplicated."
        else:
            Head._all_managers.append(story_manager)

    @staticmethod
    def get_manager(title:str)->StoryManager:
        for manager in Head._all_managers:
            if title==manager.title:
                return manager
        raise f"There is no StoryManager with the title: {title}."

    @staticmethod
    def get_titles()->List[str]:
        return [manager.title for manager in Head._all_managers]

    @staticmethod
    def get_story(full_path:str)->Story:
        split = full_path.split('/')
        title = "/".join(split[:-1])
        story_name = split[-1]
        story_manager = Head.get_manager(title)
        story = story_manager.get_story(story_name=story_name)
        return story
    

    @staticmethod
    def get_story_meta(story_or_path: Union[Story, str])->StoryMeta:
        if isinstance(story_or_path, Story):
            story = story_or_path
        else:
            story = Head.get_story(full_path=story_or_path)
        
        story_meta = StoryMeta(
            title=story.parent,
            name=story.name,
            render_name=story.render.__name__,
            docs=get_docs(story.render),
            type_hints=get_type_hints(story.render)
        )
        return story_meta


In [208]:
title = "Example/String"

story_manager = StoryManager(
    title=title
)

In [209]:
def render(arg1: int=1, arg2: int=2) -> int:
    """
    Some docs string.
    """    

    return arg1 + arg2

story_manager.create_story(
    story_name="Join",
    render=render,
)

### _Typehint Extraction Develop



In [158]:
Head.get_manager(title).stories[0]

Story(name='Join', render=<function render at 0x7f219f1912d0>, meta='', parent='Example/String')

In [108]:
def render(arg1:int, arg2:int)->int:
    return arg1 + arg2

In [109]:
import inspect
from typing import get_type_hints

class Render():
    def __init__(self):
        self.name = "John"
        
    def greet(self)->str:
        return "Hi."

def render(arg1: int, arg2: int) -> int:
    """
    Some docs string.
    """    

    return arg1 + arg2

# 함수 시그니처 얻기
signature = inspect.signature(render)

# 인자 이름과 타입 출력
print("Function parameters:")
for name, param in signature.parameters.items():
    print(f"{name}: {param.annotation}")

# 반환 타입 출력
print("\nReturn type:")
print(signature.return_annotation)


# get_type_hints를 사용하여 더 간결한 방식으로 모든 타입 힌트 얻기
type_hints = get_type_hints(render)
print("\nType hints:")
print(type_hints)


# Docstring 출력
print("\nDocstring:")
print(render.__doc__)

# inspect를 사용하여 docstring 가져오기
print("\nDocstring using inspect:")
print(inspect.getdoc(render))



Function parameters:
arg1: <class 'int'>
arg2: <class 'int'>

Return type:
<class 'int'>

Type hints:
{'arg1': <class 'int'>, 'arg2': <class 'int'>, 'return': <class 'int'>}

Docstring:

    Some docs string.
    

Docstring using inspect:
Some docs string.


In [101]:
type_hints['arg1']

int

In [ ]:
print(type_hints)

{'arg1': <class 'int'>, 'arg2': <class 'int'>, 'return': <class 'int'>}


In [ ]:
signature.parameters

mappingproxy({'arg1': <Parameter "arg1: int">,
              'arg2': <Parameter "arg2: int">})

### Test Story

In [210]:
full_path = "Example/String/Join"

render = Head.get_story(full_path)

In [211]:
from pprint import pprint

pprint(Head.get_story_meta(render).model_dump())

{'docs': 'Some docs string.',
 'name': 'Join',
 'render_name': 'render',
 'title': 'Example/String',
 'type_hints': {'arg1': 'int', 'arg2': 'int', 'return': 'int'}}


In [212]:


story_docs_template = '''
## {title}/{name}

Render Name : 
    {render_name}

Documentation :
{docs}

Type Hints :
{type_hints}

Result:
    {result}

'''

In [219]:
import json

story_meta = Head.get_story_meta(render).model_dump()
story_meta['type_hints'] = json.dumps(story_meta['type_hints'], indent=2)

In [223]:
def set_indent(text:str, indent:int=4):
    space = " " * indent
    split = text.split('\n')
    split = [space + line for line in split]
    return "\n".join(split)

In [224]:
story_meta['type_hints'] = set_indent(story_meta['type_hints'])


In [225]:
story_meta['docs'] = set_indent(story_meta['docs'])

In [220]:
print(story_meta['type_hints'])

{
  "arg1": "int",
  "arg2": "int",
  "return": "int"
}


In [221]:
story_meta["result"] = render.render()

In [226]:
print(story_docs_template.format(
    **story_meta
))


## Example/String/Join

Render Name : 
    render

Documentation :
    Some docs string.

Type Hints :
    {
      "arg1": "int",
      "arg2": "int",
      "return": "int"
    }

Result:
    3


